In [14]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import os.path
from os.path import join
import numpy as np
import imodelsx
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import data
files_dict = data.load_files_dict_single_site()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Kevin_AnnArbor_LosAngeles_SaltLakeCity
Jonathan_Indianapolis_Milwaukee_Pittsburgh


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Katie_Dallas_Seattle_Tucson
Spencer_Denver_Sacramento
Patty_Charlotte_Houston_Portland
Corina_Buffalo_Cincinnati_SanFrancisco
Manish_Atlanta_Columbus_WashingtonDC
success: 0
fail: ['Los Angeles COMPLETE', 'Salt Lake City', 'Ann Arbor', 'Pittsburgh', 'Milwaukee', 'Indianapolis', 'Tucson', 'Dallas', 'Seattle', 'Denver', 'Sacramento', 'Houston', 'Charlotte', 'Portland', 'Cincinnati', 'San Francisco', 'Buffalo', 'Columbus', 'WashingtonDC', 'Atlanta']


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


### Filter data for single-site analysis

In [15]:
k = 'Atlanta'
# k = 'Columbus'
df = files_dict[k]
theme_index = np.where(
    np.array(list(map(str.lower, df.columns.values))) == 'theme')[0][0]
col_vals = df.columns[4: theme_index]

# separate into relevant pieces
qs = df['Subcategory']
responses_df = df[col_vals]
themes_df = df[df.columns[theme_index:]]

In [16]:
question, responses, theme_dict = data.get_data_for_question_single_site(
    question_num=8, qs=qs, responses_df=responses_df, themes_df=themes_df)

/home/chansingh/pedidose-efic-analysis/notebooks/data.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses[i] = np.nan
/home/chansingh/pedidose-efic-analysis/notebooks/data.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses[i] = resp.strip(' .(),')


In [17]:
def numbered_list(classes):
    # return '\n'.join([f'Class {i+1}. {c}' for i, c in enumerate(classes)])
    return '\n'.join([f'{i+1}. {c}' for i, c in enumerate(classes)])


classification_prompt = '''### You are given a question, response, and a numbered list of themes below.

Question: {question}

Response: {response}

Themes:
{classes_as_numbered_list}

### Which of the themes does the response above belong to? Return the theme number.

Answer:'''

llm = imodelsx.llm.get_llm('gpt-4', repeat_delay=3)

In [18]:
response = responses[1]
if len(theme_dict) > 1:
    assert not pd.isna(response)
    prompt = classification_prompt.format(
        question=question,
        classes_as_numbered_list=numbered_list(theme_dict.keys()),
        response=response,
    )
    print(prompt)
# llm(prompt)

### You are given a question, response, and a numbered list of themes below.

Question: Would you want somebody else to decide if your child could be a research subject?  If so, who?

Response: Professionals

Themes:
1. Medical professionals
2. Spouse
3. Spouse or other close family
4. Would not consent

### Which of the themes does the response above belong to? Return the theme number.

Answer:


In [29]:
# screen valid questions
valid_question_nums = []
for question_num in tqdm(range(len(qs)), position=0):

    question, responses, theme_dict = data.get_data_for_question_single_site(
        question_num=question_num, qs=qs, responses_df=responses_df, themes_df=themes_df)

    # print(theme_dict)
    # valid only if there are multiple themes
    if len(theme_dict) > 1:

        # valid only if the number of theme values == number of non-null responses
        if sum(theme_dict.values()) == sum(responses.notna()):
            valid_question_nums.append(question_num)
        # assert sum(theme_dict.values()) == sum(responses.notna(
        # )), f'{sum(theme_dict.values())} != {sum(responses.notna())}, theme_dict: {theme_dict}, responses: {responses}\nquestion {question_num}: {question}'
print('num valid qs', len(valid_question_nums))

  0%|          | 0/46 [00:00<?, ?it/s]/home/chansingh/pedidose-efic-analysis/notebooks/data.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses[i] = resp.strip(' .(),')
/home/chansingh/pedidose-efic-analysis/notebooks/data.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses[i] = np.nan
100%|██████████| 46/46 [00:00<00:00, 1454.97it/s]

num valid qs 19


### Run classification

In [ ]:
classifications = defaultdict(list)
for question_num in tqdm(valid_question_nums, position=0):

    question, responses, theme_dict = data.get_data_for_question_single_site(
        question_num=question_num, qs=qs, responses_df=responses_df, themes_df=themes_df)

    for response_num in tqdm(range(len(responses)), position=1):
        response = responses.values[response_num]

        if not pd.isna(response):
            prompt = classification_prompt.format(
                question=question,
                classes_as_numbered_list=numbered_list(theme_dict.keys()),
                response=response,
            )
            ans = llm(prompt)
            classifications[question_num].append(ans)

### Compare acc

In [ ]:
def process_classifications(x):
    # note, only works with <10 classes
    for i in range(1, 10):
        if x.startswith(str(i)):
            return i
    else:
        return 1


diffs = []
class_count_pred = []
class_count_gt = []
for question_num in valid_question_nums:
    question, responses, theme_dict = data.get_data_for_question_single_site(
        question_num=question_num, qs=qs, responses_df=responses_df, themes_df=themes_df)

    answer_counts = theme_dict.values()
    class_counts = pd.Series(classifications[question_num]).apply(
        process_classifications).value_counts()

    # convert value_counts to dict
    answer_counts_dict = {i + 1: round(v)
                          for i, v in enumerate(theme_dict.values())}
    class_counts_dict = {k: round(v) for k, v in class_counts.items()}
    for i in answer_counts_dict.keys():
        if i not in class_counts_dict:
            class_counts_dict[i] = 0

    # print(answer_counts_dict)
    # print(class_counts_dict)
    # print()

    class_count_pred += list(class_counts_dict.values())
    class_count_gt += list(answer_counts_dict.values())

    diffs.append(np.sum(np.abs(np.array(list(answer_counts_dict.values())) -
                               np.array(list(class_counts_dict.values())))))

In [ ]:
# differences
# plt.hist(diffs)

# Counts as scatter plot
# plt.plot(class_count_gt, class_count_pred, 'o', alpha=0.05)
# plt.xlabel('Ground Truth Class Count')
# plt.ylabel('Predicted Class Count')

n = max(class_count_gt + class_count_pred)
mat = np.zeros((n, n))
for i, j in zip(class_count_pred, class_count_gt):
    mat[i - 1, j - 1] += 1
sns.heatmap(mat, annot=True, fmt='g', vmax=5, cmap='Blues')
plt.ylabel('Predicted class count')
plt.xlabel('Ground truth class count')

plt.savefig('../figs/classification_heatmap.pdf', bbox_inches='tight')
plt.show()